In [1]:
import os
import glob
import sys
import copy as cp

import numpy as np
import pandas as pd
idx = pd.IndexSlice
import matplotlib.pyplot as plt
import pims as pm
from pims import pipeline
import trackpy as tp
import matplotlib as mpl

sys.path.insert(0, '../icenumerics')
import icenumerics as ice
ureg = ice.ureg
import transformations as tr

sys.path.insert(0, '../Scripts/')
import support as sup
from importlib import reload  
reload(sup)

def as_gray(frame):
    red = frame[:, :, 0]
    green = frame[:, :, 1]
    blue = frame[:, :, 2]
    return 0.2125 * red + 0.7154 * green + 0.0721 * blue

os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/bin/ffmpeg"

%load_ext autoreload
%autoreload 2

from tqdm import tqdm
tqdm.pandas()

/home/carolina/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Processing videos

In this script, I will analyze all the (a set of) frames of the video. I will store, the vertices dataframe, the histogram of the particles and a snapshot of the sistem with the vertex assignation overimposed. 

In [2]:
directory =  "/media/Carol/Dynamic/Experiments/Colloidal_Ice/Cairo_lattice/Analyzed_videos_h=2-w=2"

## Introduce your parameters

In [3]:
# < Looking for files .avi and open those videos with pims >
filenames = [file for file in glob.glob(
    os.path.join(directory,('*.avi')))]

filenames = [f for f in filenames]
exp = 0
fname = filenames[exp]
video = pm.open(fname)

/home/carolina/anaconda3/lib/python3.7/site-packages/pims/api.py:207: UserWarning: <class 'pims.pyav_reader.PyAVReaderTimed'> errored: 'NoneType' object has no attribute 'open'
  warn(message)
/home/carolina/anaconda3/lib/python3.7/site-packages/pims/api.py:207: UserWarning: <class 'pims.pyav_reader.PyAVReaderIndexed'> errored: 'NoneType' object has no attribute 'open'
  warn(message)
Exception ignored in: <function PyAVReaderIndexed.__del__ at 0x7f481dc6dd40>
Traceback (most recent call last):
  File "/home/carolina/anaconda3/lib/python3.7/site-packages/pims/pyav_reader.py", line 368, in __del__
    self._container.close()
AttributeError: 'NoneType' object has no attribute 'close'


In [4]:
# < Parameters obtained in script 1_Align_Experiment_and_ColloidalIce. >

dataset = pd.DataFrame(data = {"scale":1.7,"x_offset":-83,"y_offset":392,"angle":0}, index=[0])
dataset

,scale,x_offset,y_offset,angle
0,1.7,-83,392,0


## Functions used in the analysis

In [5]:
# < This function uses trackpy package to track particles, the parameters can be tuned to optimize tracking. >
def locate(filename):

    return tp.locate(as_gray(filename),  2*round(radius)+1, minmass = 100,topn = 73, threshold = 0)

In [6]:
def match_colloid_frame(col,trj_frame, x1, y1, width, height):

    video_shape = [height,width]
    box = mpl.patches.Rectangle(((x1),(y1)),(width),(height),angle = 1.7, alpha = 1,facecolor='none',edgecolor='r',linewidth=2)
    
    remove = []
    
    for i,c in enumerate(col):
        
        if (c.center[0].magnitude > x1) and (c.center[0].magnitude < (x1 + width)) and (c.center[1].magnitude > y1) and (c.center[1].magnitude < (y1 + height)):
            d = np.sqrt((trj_frame.x-c.center[0].magnitude)**2 + (trj_frame.y-c.center[1].magnitude)**2)
            closest = d.idxmin()
            closest_part = trj_frame.loc[idx[closest],:]
            units = c.center.units
            c.colloid = [(closest_part.x-c.center[0].magnitude),(closest_part.y-c.center[1].magnitude),0]*units
            if c.direction@c.colloid<0:
                c.direction = -c.direction 
        else:
            remove.append(i)
            
            
    col.remove(remove)            
    return col

In [7]:
def particle_locations_to_colloids(locations):
    """ converts locations into a colloidal ice """
    entry = dataset.loc[0]
    square_spins = ice.spins()

    square_spins.create_lattice("square",[nx,ny],lattice_constant = 30*ureg.um, border = "periodic" )

    square_spins = tr.rotate(square_spins,(entry[["angle"]].values[0])*ureg.deg)
    square_spins = tr.scale(square_spins,entry[["scale"]].values[0]*np.array([1,1]))
    square_spins = tr.translate(square_spins,entry[["x_offset","y_offset"]].values*ureg.um)
    
    col = ice.colloidal_ice(square_spins, particle, trap, height_spread = 0, susceptibility_spread = 0.1)

    col = match_colloid_frame(col,locations, x1, y1, width, height)
            
    return col

In [8]:
def particle_locations_to_colloids(locations):
    
    entry = dataset.loc[0]
    
    nx = 5
    ny = 5
    a = 19.5458*ureg.um
    
    particle = ice.particle()

    trap_long = ice.trap(trap_sep = 10*ureg.um,
                   height = 30*ureg.pN*ureg.nm,
                   stiffness = 0.1*ureg.fN/ureg.nm)

    trap_short = ice.trap(trap_sep = 4.5258*ureg.um,
                   height = 30*ureg.pN*ureg.nm,
                  stiffness = 0.1*ureg.fN/ureg.nm)
    
    traps = []

    sp_cairo = sup.spins()
    
    #display(entry)
    
    sp_cairo.create_lattice(geometry = "cairo",size =[nx,ny],lattice_constant=a, border="periodic")
    
    for s in sp_cairo:
    
        if ( s.direction[0].magnitude == a.magnitude or s.direction[0].magnitude == -a.magnitude or 
            s.direction[1].magnitude == a.magnitude or s.direction[1].magnitude == -a.magnitude ):
        
            traps.append(trap_short)
        
        else:
        
            traps.append(trap_long)
            
    sp_cairo = tr.rotate(sp_cairo,(entry[["angle"]].values[0])*ureg.deg)
    sp_cairo = tr.scale(sp_cairo,entry[["scale"]].values*np.array([1,1]))
    sp_cairo = tr.translate(sp_cairo,entry[["x_offset","y_offset"]].values*ureg.um)
    
    col = ice.colloidal_ice(sp_cairo, particle, traps, height_spread = 0, susceptibility_spread = 0.1)
    
    col = match_colloid_frame(col,locations, x1, y1, width, height)
    
    return col

In [9]:
def generate_hist(col):
    
    variable_v = []
    variable_h = []

    fixed_r = []
    fixed_l = []

    for c in col:
        if (c.direction[0] == 1 or c.direction[0] == -1):
            variable_h.append(c.colloid.magnitude)

        elif (c.direction[1] == 1 or c.direction[1] == -1):
            variable_v.append(c.colloid.magnitude)

        elif (c.direction[1] > 0.5 or c.direction[1] < -0.5):
            fixed_r.append(c.colloid.magnitude)
        else:
            fixed_l.append(c.colloid.magnitude)

    var_v = np.array(variable_v)
    var_h = np.array(variable_h)
    fixed_r = np.array(fixed_r)
    fixed_l = np.array(fixed_l)

    variable = np.concatenate((var_v[:,1],var_h[:,0]))
    fixed = np.concatenate((fixed_r[:,1],fixed_l[:,0]))
    var = {'colloid': variable}
    fix = {'colloid': fixed}

    df_v = pd.DataFrame(data=var)
    df_f = pd.DataFrame(data=fix)

    
    return df_v, df_f

In [10]:
# < Define the ROI analyzed. >
x1 = 0
y1 = 0

height = 310
width = 310

video_shape = [height,width]
box = mpl.patches.Rectangle(((x1),(y1)),(width),(height),angle = 0, alpha = 1,facecolor='none',edgecolor='r',linewidth=5)

## Starting loop

In [11]:
v_df = []
frames = []
trap_variable = []
trap_fixed = []

for frame in range(0,len(video),100):


    # < Defining diameter for the tracking >
    diameter = 10
    radius = diameter/2

    loc = locate(video[frame])
    if len(loc.x) != 73:
        print('loosing_particles at frame'+str(frame))
    col =  particle_locations_to_colloids(loc)
    v = ice.vertices()
    v = v.colloids_to_vertices(col)
    variable, fixed = generate_hist(col)
    
    v_df.append(v.vertices.copy())
    trap_variable.append(variable.copy())
    trap_fixed.append(fixed.copy())
    
    frames.append(frame)

#     f, (ax1) = plt.subplots(1,1,figsize = (20,20))

#     ax1.imshow(video[frame], alpha = 1)
#     v.display(ax1)
#     plt.axis('off')
#     f.savefig('Analyzed_'+str(frame))
#     plt.close()

v_def = pd.concat(v_df,keys=frames, names = ["frame"])
trap_v = pd.concat(trap_variable,keys=frames, names = ["frame"])
trap_f = pd.concat(trap_fixed,keys=frames, names = ["frame"])

v_def.to_csv(os.path.join(directory,"vertices_h=2_w=2_"+str(exp)+".dat"), sep='\t')
trap_v.to_csv(os.path.join(directory,"hist_variable_trap_h=2_w=2_"+str(exp)+".dat"), sep='\t')
trap_f.to_csv(os.path.join(directory,"hist_fixed_trap_h=2_w=2_"+str(exp)+".dat"), sep='\t')

../Scripts/support.py:283: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY
/home/carolina/CairoLattice_ACI_PROJECT/Experiments/transformations.py:39: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  [0,0,1]])
